In [1]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Overlapping Generation Model (OLG)

## Applying the OLG model to examine the effects of a social security system

In this project, we examine the effects of a social security system through an Overlapping Generation Model (OLG). First, we set up the "baseline" model, which does not include a social security system. Here we analyze the steady state of capital and how time preferences affect capital accumulation. In the baseline model, we also show how to analytically solve the model and next how to solve it numerically with calibrated model parameters. 

Next, we extend our model to include a Pay-As-You-Go (PAYG) social security system. The PAYG system where the government raises taxes from the young working population and transfers them to the current old. Here we examine how different tax rates affect the capital accumulation and steady state values. 

We find that the introduction of a PAYG social security system decreases the steady state capital. This decrease in steady state capital happens through two channels: 1) When the tax rate increases the young have a lower disposable income and need to save less to obtain the same level of consumption in period 1. 2) When the tax rate increases the old receive higher level of social benefit. This means that they can (and will) save less when young, and still obtain the same level of consumption in period 2.

## Model setup

In this setup the economy is populated by agents that live for 2 periods agents i.e. $t = 1,2$. $L_t$ individuals are born in period t and the population is assumed to grow at rate n:

$$ L_t = L_{t-1}(1+n) $$

We assume agents derive utility from consumption while alive and log-preferences:

$$ U_t = u(c_{1t}) + \frac{1}{1+\rho} u(c_{2t+1}) $$

where 
* $c_{1t}$ denotes consumption when the agent is young at time t
* $c_{2t+1}$ denotes consumption when the agent is old at time t+1
* $\rho$ is the discount rate
* $u(x) = \log{x}$

While young, agents supply 1 unit of labor inelastically at wage rate $w_t$ which is split into consumption $c_{1t}$ and savings $s_t$. When old they consume their gross savings where the interest rate earned from $t$ to $t+1$ is given by $r_{t+1}$. Thus, agents face the following budget constraints:

$$ c_{1t} + s_t = w_t $$
$$ c_{2t+1} = (1+r_{t+1}) s_t $$

It follows that the lifetime budget constraint is given by:

$$ c_{1t} + \frac{c_{2t+1}}{1+r_{t+1}} = w_t $$

The firms of the economy is assumed to be characterized by Cobb-Douglas production technology where the inputs are used are capital $K$ and labor $L$ to obtain output $Y$:

$$ f(K,L) = Y = K^\alpha L^{1-\alpha},      \alpha \in (0,1) $$

## Analytical solution

### Households' problem

In this section we set up the households' problem. Ultimately we are interested in the agents optimal savings behaviour which defines the capital accumulation. First, we define lifetime utility:

In [2]:
#Initialize variabels in Sympy
c1 = sm.symbols('c_1t')
c2 = sm.symbols('c_2t+1')
rho = sm.symbols('rho')

#Setup utility
uc1 = sm.ln(c1)
uc2 = sm.ln(c2)
U = uc1 + 1/(1+rho) * uc2
display(U)

log(c_1t) + log(c_2t+1)/(rho + 1)

Next, we setup the lifetime budget constraint:

In [3]:
#Initialize variabels in Sympy
wt = sm.symbols('w_t')
rt1 = sm.symbols('r_t+1')

#Setup lifetime budget constraint
bc = c1 + c2 / (1+rt1) - wt
display(bc)

c_1t + c_2t+1/(r_t+1 + 1) - w_t

Setting up the Langrangian: 

In [4]:
#Initialize variabels in Sympy
lamb = sm.symbols('lambda')

#Setup Lagrangian
L = U + lamb * (-1)*bc
display(L)

-lambda*(c_1t + c_2t+1/(r_t+1 + 1) - w_t) + log(c_1t) + log(c_2t+1)/(rho + 1)

Defining first order conditions (FOC) from the lagrangian: 

In [5]:
#First order conditions
foc1 = sm.Eq(0,sm.diff(L,c1))
foc2 = sm.Eq(0,sm.diff(L,c2))
display(foc1)
display(foc2)

Eq(0, -lambda + 1/c_1t)

Eq(0, -lambda/(r_t+1 + 1) + 1/(c_2t+1*(rho + 1)))

Solving for the lagrangian multiplier  $\lambda$:

In [6]:
lam1 = sm.solve(foc1,lamb)[0] #solving foc1 wrt. to lambda
lam2 = sm.solve(foc2,lamb)[0] #solving foc2 wrt. to lambda
display(lam1)
display(lam2)

1/c_1t

(r_t+1 + 1)/(c_2t+1*(rho + 1))

Thus, the Euler-equation is given by:

In [7]:
euler1 = sm.solve(sm.Eq(lam1,lam2),c1)[0] #Equating lambdas and solving
euler2 = sm.Eq(euler1, c1)
display(euler2)

Eq(c_2t+1*(rho + 1)/(r_t+1 + 1), c_1t)

This is the Euler equation. From the equation we see that consumers choose a level of consumption such that the marginal rate of substitution between present consumption and future consumptions equals the marginal rate of transformation.



**Optimal savings behaviour**

Next we examine the optimal savings behaviour. Replacing per period budget constraint in the Euler and solving for $s_t$ will characterize individual savings behaviour

In [8]:
#Initialize variabels in Sympy
st = sm.symbols('s_t')

#Setup per period BC
bc1 = wt-st #budget constraint for young
bc2 = (1+rt1)*st #budget constraint for old

#Replace
euler3 = euler2.subs(c1,bc1) #substituting c1 with budget constraint for young
euler4 = euler3.subs(c2,bc2) #substituting c2 with budget constraint for old
display(euler4)

Eq(s_t*(rho + 1), -s_t + w_t)

Solving for optimal savings behaviour:

In [9]:
saving = sm.solve(euler4,st)[0] #solving substituted euler equation wrt. savings
opt_saving = sm.Eq(saving, st) #equate with s_t
display(opt_saving)

Eq(w_t/(rho + 2), s_t)

The optimal savings behaviour is affected by the timepreference parameter, $\rho$, which lowers savings for higher values.

### Firms' problem

In this section we setup the firms' problem. Firms take prices as given and hence, maximize profit $\Pi$ given by cf. theory section:

$$ \Pi = K^\alpha_t L^{1-\alpha}_t - r_t K_t - w_t L_t $$

First, we setup the cobb-douglas production function:

In [10]:
#Initialize variabels in Sympy
alpha = sm.symbols('alpha')
K = sm.symbols('K_t')
L = sm.symbols('L_t')
rt = sm.symbols('r_t')
wt = sm.symbols('w_t')
Y = sm.symbols('Y')

#Setting up the cobb-douglas production function
Y = K**alpha * L**(1-alpha)
display(Y)

K_t**alpha*L_t**(1 - alpha)

Next, we derive the interest and wage rate. Under the assumption of perfectly competitive markets we know that input factors are rewarded their marginal product: 

In [11]:
#Derivatives of the production function
dyk = sm.diff(Y,K)
dyl = sm.diff(Y,L)

#Equating
interest = sm.Eq(rt,dyk) 
wage = sm.Eq(wt,dyl)
display(interest)
display(wage)

Eq(r_t, K_t**alpha*L_t**(1 - alpha)*alpha/K_t)

Eq(w_t, K_t**alpha*L_t**(1 - alpha)*(1 - alpha)/L_t)

Rewriting to pr. worker:

In [12]:
#Initialize variabels in Sympy
k = sm.symbols('k_t')

#Rewriting
intrate = sm.Eq(rt,(alpha*k**(alpha-1)))
wagerate = sm.Eq(wt, ((1-alpha)*k**(alpha)))
display(intrate)
display(wagerate)

Eq(r_t, alpha*k_t**(alpha - 1))

Eq(w_t, k_t**alpha*(1 - alpha))

### Law-of-motion and steady state capital

The aggregate capital stock in $t+1$ is given by
$$k_{t+1}(1+n) = s_t$$

Recalling the optimal savings behaviour:

In [13]:
display(opt_saving)

Eq(w_t/(rho + 2), s_t)

Substitute for the wage rate and use aggregate capital stock to obtain the law-of-motion. The law of motion shows how capital per unit of labor evolves over time:

In [14]:
#Initializing variabels in Sympy
k1 = sm.symbols('k_t+1')
n = sm.symbols('n')

#Replace wage rate in savings
savings = opt_saving.subs(wt, ((1-alpha)*k**(alpha)))

#Using definition of aggregate capital
cap = k1 * (1+n)
agcap = savings.subs(st,cap)

#Solving for k_t+1 to obtain law of-motion
lom = sm.solve(agcap,k1)[0]
lom = sm.Eq(k1,lom) #Equate
display(lom)

Eq(k_t+1, k_t**alpha*(1 - alpha)/(n*rho + 2*n + rho + 2))

Use in steady state (SS) that $k_t = k_{t+1} = k^*$. Hence, the law-of-motion is: 

In [15]:
#Initialize variabels in Sympy
kstar = sm.symbols('k^*')

#Replace with k*
lom1 = lom.subs(k1, kstar)
lom2 = lom1.subs(k, kstar)

#Solve for k*
lom3 = sm.solve(lom2, kstar)[0]
display(lom3)

((-n*rho - 2*n - rho - 2)/(alpha - 1))**(1/(alpha - 1))

We can compute the SS value by turning the law-of-motion into a python function:

In [16]:
ss_func = sm.lambdify((n, rho, alpha),lom3) #turn into python function
ss_capital = ss_func(0.2,0.5,1/3) #give the funtion random values
print(f'The SS level of capital for n=0.2, rho=0.5 and alpha=1/3 is {ss_capital}')

The SS level of capital for n=0.2, rho=0.5 and alpha=1/3 is 0.10475656017578489


## Numerical solution

We can also solve for the SS numerically:

In [17]:
#Set parameter values
alpha = 1/3 #capital income share
n = 0.2 #population growth
rho = 0.5 #timepreference

#Define objective function i.e. the law-of-motion as seen above
f = lambda k: k**alpha #input for the objective function
obj_kss = lambda kss: kss - (f(kss)*(1-alpha))/(n*rho+2*n+rho+2) 

#Optimizing using the bisect method
result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='bisect')

print('The SS level of capital for n=0.2, rho=0.5 and alpha=1/3 is',result.root)    

The SS level of capital for n=0.2, rho=0.5 and alpha=1/3 is 0.10475656017450916


## Graphical representation of dynamics

From the analytical section recall:

In [18]:
display(wagerate)
display(opt_saving)
display(cap)

Eq(w_t, k_t**alpha*(1 - alpha))

Eq(w_t/(rho + 2), s_t)

k_t+1*(n + 1)

This we use to construct the transition equation

In [19]:
def transition(k,alpha,rho,n):
    """
    Returns:
    k1(float) capital in the next period
    
    Args:
    k (float): Initial stock of capital
    alpha (float): Capital income share.
    rho (float): Timepreference parameter
    n (float): Population growth
    
    output: 
    Computes the wage rate, optimal savings and capital in next period. 
    
    """
    # Raise ValueErrors
    if n>1:
        raise ValueError("Population growth cannot exceed 1")
    if alpha>1: 
        raise ValueError("Capital income share cannot exceed 1")
    if rho < 0: 
        raise ValueError("Timepreference cannot be a negative number")
    
    w = k**alpha * (1-alpha) #wage rate
    s = w / (rho+2) #optimal savings
    k1 = s / (n+1) #capital in the next period
    return k1

Use the transition function to create all values of capital

In [20]:
def transition_curve(alpha,rho,n,T=1000,k_min=1e-20,k_max=6):
    """
    Returns:
    k_1(array): Array of capital in period 1
    k_2(array): Array of capital in period 2
    
    Args:
    alpha (float): Capital income share
    rho (float): Timepreference parameter
    n (float): Population growth
    T (int): Number of periods. Default 1000 periods.
    k_min (float): Minimum capital value. Default 1e-20.
    k_max (float): Maximum capital value. Default 6.
    
    output: 
    For every value of capital computes the value of capital in the next period using the transition equation. 
    """
    
    if n>1:
        raise ValueError("Population growth cannot exceed 1")
    if alpha>1: 
        raise ValueError("Capital income share cannot exceed 1")
    if rho < 0: 
        raise ValueError("Timepreference cannot be a negative number")
    
    #grids
    k_1 = np.linspace(k_min,k_max,T)
    k_1 = np.linspace(1e-20,6,T)
    k_2 = np.empty(T)
    
    #construct array of "tomorrows" capital
    for i,k in enumerate(k_1):
        k_plus = transition(k,alpha,rho,n)
        k_2[i] = k_plus
        
    return k_1, k_2

Next we plot the transition diagram:

In [21]:
def fig(rho):
    """
    Returns:
    Plots the transition curve
    
    Args:
    rho(float): Timepreference parameter
    """
    #parameters
    alpha = 1/3
    rho = rho
    n = 0.2
    
    #transition curve
    k_1, k_2 = transition_curve(alpha,rho,n,T=1000,k_min=1e-20,k_max=6)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(k_1,k_2, label="Transition curve") #transition curve
    ax.plot(k_1,k_1, '--', color='grey',label="45 degree") #45 degree line
    ax.set_xlabel('$k_t$')
    ax.set_ylabel('$k_t+1$')
    ax.set_title('Transition curve')
    ax.legend()
    ax.set_xlim([0,0.2])
    ax.set_ylim([0,0.2]);
    return

import ipywidgets as widgets
widgets.interact(fig, 
    rho = widgets.FloatSlider(description='rho', min=0, max=16, step=0.01, value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='rho', max=16.0, step=0.01), Output()), _dom_classes=…

As seen on the figure the steady state capital decreases as $\rho$ $\rightarrow$ $\infty$.
$\rho$ is a measurement of time preference (impatience). The higher
$\rho$ is the more impatient are the agents. Hence, they will
spend more when young which implies lower savings and thereby lower steady state capital.

# Model extension

We now impose a pay-as-you-go social security system in the model. The government raises taxes, $\tau$, from the young population in period t and transfers them to the old. Budget constraints is thus:

$$ c_{1t} + s_t + \tau w_t = w_t $$

$$ c_{2t+1} = (1+r_{t+1}) s_t + \tau w_t $$

In [22]:
#Initializing symbols
tau = sm.symbols('tau')
c1 = sm.symbols('c_1t')
c2 = sm.symbols('c_2t+1')
rho = sm.symbols('rho')
alpha = sm.symbols('alpha')
K = sm.symbols('K_t')
L = sm.symbols('L_t')
rt = sm.symbols('r_t')
wt = sm.symbols('w_t')
Y = sm.symbols('Y')
lamb = sm.symbols('lambda')
st = sm.symbols('s_t')
kstar = sm.symbols('k^*')
k1 = sm.symbols('k_t+1')
n = sm.symbols('n')
k = sm.symbols('k_t')

#Lifetime budget constraint
life_con = c1+c2/(1+rt1)-tau/(1+rt1)*wt-(1-tau)*wt

In [23]:
#Setting up the new lagrangian
L1 = U + lamb * (-1)*life_con
L1

-lambda*(c_1t + c_2t+1/(r_t+1 + 1) - tau*w_t/(r_t+1 + 1) - w_t*(1 - tau)) + log(c_1t) + log(c_2t+1)/(rho + 1)

In [24]:
#First order conditions
foc3 = sm.Eq(0,sm.diff(L1,c1))
foc4 = sm.Eq(0,sm.diff(L1,c2))
display(foc3)
display(foc4)

Eq(0, -lambda + 1/c_1t)

Eq(0, -lambda/(r_t+1 + 1) + 1/(c_2t+1*(rho + 1)))

In [25]:
#Solving the FOC's
lam1 = sm.solve(foc3,lamb)[0]
lam2 = sm.solve(foc4,lamb)[0]
display(lam1)
display(lam2)

1/c_1t

(r_t+1 + 1)/(c_2t+1*(rho + 1))

In [26]:
#Equate lambdas and solve for Euler eq.
euler1 = sm.solve(sm.Eq(lam1,lam2),c1)[0] 
euler2 = sm.Eq(euler1, c1)
display(euler2)

Eq(c_2t+1*(rho + 1)/(r_t+1 + 1), c_1t)

In [27]:
#Substituting per period budgetconstraint
sav = euler2.subs(c2,((1+rt1)*st+tau*wt))
sav2 = sav.subs(c1,(wt-st-tau*wt))

#Solving for savings
opt_sav = sm.solve(sav2,st)[0]
opt_sav

w_t*(-r_t+1*tau + r_t+1 - rho*tau - 2*tau + 1)/(r_t+1*rho + 2*r_t+1 + rho + 2)

In [28]:
sav1 = opt_sav.subs(rt1,(alpha*k**(alpha-1)))
sav2 = sav1.subs(wt,(1-alpha)*k**(alpha))
sav_func = sm.lambdify((k,alpha,tau,rho),sav2)

Next we define a new transition function for the new savings function:

In [29]:
def transition_payg(k,alpha,rho,tau,n):
    """
    Returns:
    k1_payg(float) capital in the next period
    
    Args:
    k (float): Initial stock of capital
    alpha (float): Capital income share.
    rho (float): Timepreference parameter
    n (float): Population growth
    tau (float): Taxrate
    
    output: 
    Computes the wage rate, optimal savings and capital in next period. 
    
    """
    if n>1:
        raise ValueError("Population growth cannot exceed 1")
    if alpha>1: 
        raise ValueError("Capital income share cannot exceed 1")
    if rho < 0: 
        raise ValueError("Timepreference cannot be a negative number")
    if tau > 1:
        raise ValueError("Taxrate cannot exceed 1")
    
    w = k**alpha * (1-alpha)
    s = sav_func(k,alpha,tau,rho)
    k1_payg = s / (n+1)
    return k1_payg

In [30]:
def transition_curve_payg(alpha,rho,n,tau,T=1000,k_min=1e-20,k_max=6):
    """
    Returns:
    k_1_payg(array): Array of capital in period 1
    k_2_payg(array): Array of capital in period 2
    
    Args:
    alpha (float): Capital income share
    rho (float): Timepreference parameter
    n (float): Population growth
    tau (float): Taxrate
    T (int): Number of periods. Default 1000 periods.
    k_min (float): Minimum capital value. Default 1e-20.
    k_max (float): Maximum capital value. Default 6.
    
    output: 
    For every value of capital computes the value of capital in the next period using the transition equation. 
    """
    #grids
    k_1_payg = np.linspace(k_min,k_max,T)
    k_2_payg = np.empty(T)
    
    #construct array of "tomorrows" capital
    for i,k in enumerate(k_1_payg):
        k_plus = transition_payg(k,alpha,rho,tau,n)
        k_2_payg[i] = k_plus
        
    return k_1_payg, k_2_payg

In [31]:
def fig_payg(tau):
    """
    Returns:
    Plots the transition curve with and without a PAYG social security system
    
    Args:
    tau(float): Taxrate
    """
    
    #parameters
    tau = tau
    alpha = 1/3
    rho = 0.5
    n = 0.2

    #transition curves
    k_1_payg, k_2_payg = transition_curve_payg(alpha,rho,n,tau,T=1000,k_min=1e-20,k_max=6)
    k_1, k_2 = transition_curve(alpha,rho,n,T=1000,k_min=1e-20,k_max=6)
    
    #plot
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(k_1_payg,k_2_payg,label="PAYG")
    ax.plot(k_1,k_2,label="Baseline")
    ax.plot(k_1_payg,k_1_payg, '--', color='grey')
    ax.set_xlabel('$k_t$')
    ax.set_ylabel('$k_t+1$')
    ax.set_title('Transition curve')
    ax.set_xlim([0,0.2])
    ax.set_ylim([0,0.2]);
    ax.legend()
    return

import ipywidgets as widgets
widgets.interact(fig_payg, 
    tau = widgets.FloatSlider(description='tau', min=0, max=1, step=0.01, value=0.3),
);

interactive(children=(FloatSlider(value=0.3, description='tau', max=1.0, step=0.01), Output()), _dom_classes=(…

We see that when the tax rate increases the steady state capital decreases. This decrease in steady state capital happens through two channels:
1) When the tax rate increases the young have a lower disposable income and need to save less to obtain same level of consumption in period 1.
2) When the tax rate increases the old receive higher level of social benefit. This means that they can (and will) save less when young, and still obtain the same level of consumption in period 2.

# Numerical optimization

We want to optimize capital and solve for $w^*$, $s^*$,$c_{1}^*$, $c_{2}^*$ and $u^*$:

In [32]:
def save_func(k,alpha,tau,rho):
    """
    Returns:
    Optimal savings
    
    args:
    k(float): Capital
    
    output:
    Computes the optimal savings for calibrated parameters and initial capital stock
    """
    sav = (k**alpha*(1-alpha)*(-alpha*k**(alpha-1)*tau+alpha*k**(alpha-1)-rho*tau-2*tau+1)/(alpha*k**(alpha-1)*rho+2*alpha*k**(alpha-1)+rho+2)-k)
    return sav

def obj(k,alpha,tau,rho):
    return -save_func(k,alpha,tau,rho)

def optimizer(obj,k,alpha,tau,rho):
    res = optimize.minimize_scalar(obj,method="bounded",bounds=(0,1),args=(rho,alpha,tau))
    return res

In [33]:
def ss_vals(obj,k,alpha,tau,rho,n):
    """
    Returns:
    Steady state values
    
    Args:
    obj(func): Objective function to maximize
    k(float): Initial level of capital
    alpha(float): Capital income sharee
    tau(float): Taxrate
    rho(float): Timepreference parameter
    n(float): Population growth
    
    output:
    Computes the steady state values with the optimal capital level
    """
    
    nice = optimizer(obj,k,alpha,tau,rho)
    kstar = nice.x
    wstar = kstar**alpha * (1-alpha)
    sstar = kstar/(1+n)
    c1_star = wstar-sstar-tau*wstar
    c2_star = (1+(alpha*kstar**(alpha-1)))*sstar+tau*wstar
    u_star = c1_star + 1/(1+rho) * c2_star
    
    print(f'Steady state capital is {kstar:.5f}')
    print(f'wage in steady state is {wstar:.5f}')
    print(f'Savings in steady state is {sstar:.5f}')
    print(f'In steady state the young consume {c1_star:.5f}')
    print(f'In steady state the old consume {c2_star:.5f}')
    print(f'The utility is {u_star:.5f}')
    return

Steady state values in the baseline model: 

In [34]:
ss_vals(obj,k,1/3,0,0.5,0.2)

Steady state capital is 0.00538
wage in steady state is 0.11681
Savings in steady state is 0.00448
In steady state the young consume 0.11233
In steady state the old consume 0.05315
The utility is 0.14776


Steady state values in the extended model:

In [35]:
ss_vals(obj,k,1/3,0.3,0.5,0.2)

Steady state capital is 0.00392
wage in steady state is 0.10508
Savings in steady state is 0.00326
In steady state the young consume 0.07030
In steady state the old consume 0.07857
The utility is 0.12268


# Conclusion

As expected the inclusion of a PAYG social security system decreases the steady state capital level for realistic parameter values. Furthemore, we confirm that with a social security system in place the young save less because they can obtain the same level of consumption as old. However, the overall welfare is lower in the extended model because the disposable income is lower which decreases consumption as young. 